<a href="https://colab.research.google.com/github/ferkrum/web-scraper-clima-atual-vc-clima-historico/blob/main/Web_Scraper_Clima_Atual_vs_Clima_Hist%C3%B3rico_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Web Scraper Clima Atual vs Clima Histórico

Como saber se a temperatura atual está acima ou abaixo da média? E a umidade? 

Para responder essa pergunta usei duas fontes de dados para obter essas informações e gerar essas comparações. 

Fonte de dados: 

URL = 'https://www.climatempo.com.br/previsao-do-tempo/agora/cidade/366/santamaria-rs'

URL2 = 'https://pt.climate-data.org/america-do-sul/brasil/rio-grande-do-sul/santa-maria-4498/#climate-graph'


Linguagem: Python

_libs: requests; Beautifulsoup, datetime, pytz timezone_

.

.

_...Próximos features:_ 
* criar um bot no telegram para receber essa informação.


In [ ]:
import requests 
from bs4 import BeautifulSoup as bs

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from datetime import datetime
from pytz import timezone 
brasil = timezone('Brazil/East')
bs_time = datetime.now(brasil)
timestampBrasil = bs_time.strftime('%Y-%m-%d_%H-%M-%S') 
#print(bs_time)
mes = bs_time.strftime('%m')
mes = int(mes)

Obtendo informações:

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}

#1ª fonte de dados: dados atualizados (Clima Tempo)

---

In [ ]:
URL = 'https://www.climatempo.com.br/previsao-do-tempo/agora/cidade/366/santamaria-rs'

In [ ]:
response = requests.get(URL, headers = headers).content

In [ ]:
soup = bs(response, 'html.parser')

In [ ]:
#obtem informacoes
temperaturaAtual = soup.find("div", class_= "_flex _justify-center _align-center").get_text(' ', strip = True)
descricaoAtual = soup.find("span", class_= "col").text

In [ ]:
#para obter sensacao termica atual
descricaoAtual2 = soup.find("div", class_= "no-gutters -gray _flex _justify-center _margin-t-20 _padding-b-20 _border-b-light-1")

In [ ]:
sensacaoAtual = descricaoAtual2.find("span", class_ = "").text

In [ ]:
detalhesAtual = soup.find("ul", class_= "variables-list _border-b-light-1").get_text(' ', strip = True)

In [ ]:
detalhesAtual

'Vento W - 15km/h Umidade 66% Pressão 1022hPa'

In [ ]:
detalhesAtual = detalhesAtual.replace('Vento', '').replace('Umidade', ',').replace('Pressão', ',').strip().split(',')
ventoAtual = detalhesAtual[0].strip()
umidadeAtual = detalhesAtual[1].strip()
pressaoAtual = detalhesAtual[2].strip()

#Criando Pandas Dataframe:
(Com dados do primeiro soup.find)

1. criar um dicionário com Arrays;
2. transformar em pd.DataFrame(dicionario)


In [ ]:
print ('Santa Maria - \t', timestampBrasil)
print('Temperatura:\t', temperaturaAtual)
print('Sensação:\t', sensacaoAtual[-3:])
print('Descrição:\t', descricaoAtual)
print('Vento:\t\t',ventoAtual)
print('Umidade:\t',umidadeAtual)
print('Pressão:\t',pressaoAtual)

Santa Maria - 	 2022-07-17_12-59-55
Temperatura:	 11º
Sensação:	 11°
Descrição:	 Céu encoberto
Vento:		 W - 15km/h
Umidade:	 66%
Pressão:	 1022hPa


In [ ]:
dictCidade = {'Vento' : [ventoAtual],
              'Umidade' : [umidadeAtual],
              'Pressão' : [pressaoAtual]}

In [ ]:
df_dictCidade = pd.DataFrame(dictCidade)
df_dictCidade

,Vento,Umidade,Pressão
0,W - 15km/h,66%,1022hPa


#2ª fonte de dados: dados históricos

---


Adicionar comparativo com histórico de temperatura obtido em: 

https://pt.climate-data.org/america-do-sul/brasil/rio-grande-do-sul/santa-maria-4498/#climate-graph

In [ ]:
URL2 = 'https://pt.climate-data.org/america-do-sul/brasil/rio-grande-do-sul/santa-maria-4498/#climate-graph'

In [ ]:
response2 = requests.get(URL2, headers = headers).content

In [ ]:
soup2 = bs(response2, 'html.parser')

In [ ]:
tabela = soup2.find("table", id="weather_table").find_all("tr")

modelo do dicionario para Pandas DataFrame: 

```
df_dictCidadeHistorico = { "tempMedia" : [24.4, 24,1],
"tempMinima" : [12, 14],
"nomes" : ['janeiro', 'fevereiro']
}
```



In [ ]:
tabela

In [ ]:
len(tabela)

8

In [ ]:
mesesTitulo = []
for i in tabela[0].find_all("th"):
  mesesTitulo.append(i.get_text(' ', strip=True))
mesesTitulo.pop(0)
mesesTitulo

colunasTabela = len(tabela[0].find_all("th"))
colunasTabela


linhasTabela = len(tabela) 
dadosTitulo = []
dadosTabela = []

for i in range(1, linhasTabela):   #percorre as linhas da tabela
  linhaTabela = []
  #print("===")
  tituloLinha = tabela[i].find_all("td")[0].get_text(' ', strip=True)
  #print(tituloLinha)
  dadosTitulo.append(tituloLinha)
  linhaTabela.append(tituloLinha)

  for j in range(1, colunasTabela):
    
    
    conteudoCelula = tabela[i].find_all("td")[j].get_text(' ', strip=True).replace('&percnt','')
    #print(conteudoCelula, ' linha', i, 'coluna', j)
    try:
      linhaTabela.append(int(conteudoCelula)) #armazena os dados das celulas no array linhaTabela
    except:
      linhaTabela.append(float(conteudoCelula)) #armazena os dados das celulas no array linhaTabela

  dadosTabela.append(linhaTabela)     #armazena o array linhaTabela no array dadosTabela

mesesTitulo.insert(0,'Meses')
dadosTabela.append(mesesTitulo)


In [ ]:
dadosTabela

In [ ]:
dictTemperaturas = {}

In [ ]:
for i in dadosTabela:
  indice = i.pop(0)
  dictTemperaturas[indice] = i

In [ ]:
dictTemperaturas

In [ ]:
df_dictTemperaturas = pd.DataFrame(dictTemperaturas)

In [ ]:
df_dictTemperaturas

## Agora, finalmente, podemos comparar o valor atual com os valores históricos obtidos:
**bold text**

In [ ]:
print ('Santa Maria - \t', timestampBrasil)
print('Temperatura:\t', temperaturaAtual)
print('Sensação:\t', sensacaoAtual[-3:])
print('Descrição:\t', descricaoAtual)
print('Vento:\t\t',ventoAtual)
print('Umidade:\t',umidadeAtual)
print('Pressão:\t',pressaoAtual)


Santa Maria - 	 2022-07-17_12-59-55
Temperatura:	 11º
Sensação:	 11°
Descrição:	 Céu encoberto
Vento:		 W - 15km/h
Umidade:	 66%
Pressão:	 1022hPa


In [ ]:
consultaHistorico = df_dictTemperaturas.iloc[mes - 1]

# Resultados

In [ ]:
consultaHistorico

Temperatura média (°C)      13.1
Temperatura mínima (°C)      8.9
Temperatura máxima (°C)     18.2
Chuva (mm)                   149
Umidade(%)                    82
Dias chuvosos (d)              7
Horas de sol (h)             6.4
Meses                      Julho
Name: 6, dtype: object

In [ ]:
consultaHistorico['Umidade(%)']

82

In [ ]:
print ('A temperatura atual é',temperaturaAtual)
print ()

A temperatura atual é 11º



In [ ]:
if ( ((float(temperaturaAtual[:-1])) > float(consultaHistorico['Temperatura mínima (°C)'])) &  ((float(temperaturaAtual[:-1])) < float(consultaHistorico['Temperatura máxima (°C)'])) ):
  print('A temperatura atual está dentro da média histórica.', end='\n\n')
else: 
  print('A temperatura atual está fora da faixa histórica:\n\n')

print('Temp atual:', temperaturaAtual)
print('Temp mínima histórica:', consultaHistorico['Temperatura mínima (°C)'])
print('Temp máxima histórica:', consultaHistorico['Temperatura máxima (°C)'], end='\n\n')
print('Umidade atual:',umidadeAtual)
print('Umidade média histórica:',consultaHistorico['Umidade(%)'])




A temperatura atual está dentro da média histórica.

Temp atual: 11º
Temp mínima histórica: 8.9
Temp máxima histórica: 18.2

Umidade atual: 66%
Umidade média histórica: 82


In [ ]:
if ( ((float(umidadeAtual[:-1])) > float(consultaHistorico['Temperatura mínima (°C)'])) &  ((float(temperaturaAtual[:-1])) < float(consultaHistorico['Temperatura máxima (°C)'])) ):
  print('A temperatura atual está dentro da média histórica.')
else: 
  print('A temperatura atual está fora da faixa histórica:\n\n')
  print('Temp atual:', temperaturaAtual)
  print('Temp mínima histórica:', consultaHistorico['Temperatura mínima (°C)'])
  print('Temp máxima histórica:', consultaHistorico['Temperatura máxima (°C)'])
